In [1]:
import numpy as np
import pandas as pd
from random import sample

from coremdlr.config import defaults
from coremdlr.datasets.utils import infer_test_wells

from coremdlr.models import FeaturePredictor, LambdaModel
from coremdlr.datasets import WellLoader, FaciesDataset

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3

In [2]:
import hyperopt
from hyperopt import hp
from hyperopt.pyll.base import scope
from sklearn.metrics import f1_score, log_loss
from sklearn.utils.class_weight import compute_sample_weight

In [6]:
fmodel_args = {
    'logs': {
        'model': 'LambdaModel',
        'model_args': {'feature': 'logs'}
    }
}

XGB_SEARCH_SPACE = {
    'model_type' : 'XGB',
    'max_depth' : scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators' : scope.int(hp.quniform('n_estimators', 10, 1000, 1)),
    'objective' : 'multi:softprob',
    'n_jobs' : 2,
    'gamma' : hp.uniform('gamma', 0, 0.5),
    'subsample' : hp.uniform('subsample', 0.3, 1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.3, 1.0),
    'colsample_bylevel' : 1,
    'reg_alpha' : 0,                                    # L1 penalty
    'reg_lambda' : hp.uniform('reg_lambda', 0.1, 10),   # L2 penalty
    'tree_method' : 'gpu_hist',
}

def make_params_usable(params):
    """
    Convert params returned by hyperopt to usable `model_args`
    """
    return {
        **XGB_SEARCH_SPACE,
        **params,
        **{'max_depth' : int(params['max_depth']),
           'n_estimators' : int(params['n_estimators'])}
    }

In [7]:
def run_experiment(num_train, max_evals=10, search_space=XGB_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=['logs']) for s in train_sets]
    
    for dset in datasets:
        dset.load_or_generate_data()
    
    def train_model(model_config):
        
        log_losses = []
        
        for dset in datasets:
            
            model = FeaturePredictor(dset, model_args=model_config, feature_model_args=fmodel_args)
            model.fit(dset, verbose=False)
            print('F1 score:', f1_score(dset.y_test, model.predict(dset.X_test), average='macro'))
            
            log_losses.append(log_loss(dset.y_test, model.predict_proba(dset.X_test), 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return np.mean(log_losses)
    
    return hyperopt.fmin(
        fn=train_model,
        space=XGB_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals
    )

In [8]:
best_params_3 = run_experiment(3)

print('\n\n\t Starting TRAIN_SET_6')
best_params_6 = run_experiment(6)

print('\n\n\t Starting TRAIN_SET_9')
best_params_9 = run_experiment(9)

Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13034,)), ('top', (13034,)), ('base', (13034,)), ('logs', (13034, 11))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data


...

Training model for feature:                                                   
logs                                                                          
F1 score:                                                                     
0.2441932506912401                                                            
Training model for feature:                                                   
logs                                                                          
F1 score:                                                                     
0.22658335435945462                                                           
 90%|█████████ | 9/10 [15:56<02:06, 126.43s/it, best loss: 1.8094236429642616]

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Training model for feature:                                                   
logs                                                                          
F1 score:                                                                     
0.1364135331276292                                                            
 90%|█████████ | 9/10 [16:18<02:06, 126.43s/it, best loss: 1.8094236429642616]

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



100%|██████████| 10/10 [16:21<00:00, 98.12s/it, best loss: 1.8094236429642616] 


In [9]:
best_params_3

{'colsample_bytree': 0.30979354617965793,
 'gamma': 0.4922115536384819,
 'learning_rate': 0.024466131395594698,
 'max_depth': 9.0,
 'n_estimators': 180.0,
 'reg_lambda': 2.4542249504639906,
 'subsample': 0.8136015679487056}

In [5]:
best_params_3 = {'colsample_bytree': 0.715782636188357,
 'gamma': 0.38031439765997715,
 'learning_rate': 0.13285585695446292,
 'max_depth': 3.0,
 'n_estimators': 107.0,
 'reg_lambda': 7.063383010063417,
 'subsample': 0.5627460184070672}

In [10]:
best_params_6

{'colsample_bytree': 0.7714484413087108,
 'gamma': 0.13548335445702325,
 'learning_rate': 0.1880310114631697,
 'max_depth': 6.0,
 'n_estimators': 46.0,
 'reg_lambda': 3.625457697235419,
 'subsample': 0.8642175660983926}

In [6]:
best_params_6 = {'colsample_bytree': 0.5044540258165737,
 'gamma': 0.3606833915563988,
 'learning_rate': 0.020567898195521434,
 'max_depth': 8.0,
 'n_estimators': 278.0,
 'reg_lambda': 5.316977094203006,
 'subsample': 0.6810694533283663}

In [11]:
best_params_9

{'colsample_bytree': 0.8602391670037453,
 'gamma': 0.33255214554243856,
 'learning_rate': 0.0292812650855257,
 'max_depth': 7.0,
 'n_estimators': 53.0,
 'reg_lambda': 3.2474378378010935,
 'subsample': 0.889380676447874}

In [7]:
best_params_9 = {'colsample_bytree': 0.4845409209422611,
 'gamma': 0.4368302424155825,
 'learning_rate': 0.11360839815510773,
 'max_depth': 3.0,
 'n_estimators': 90.0,
 'reg_lambda': 8.283549867329606,
 'subsample': 0.9353591985992722}

In [12]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=['logs']) for s in train_sets]
    
    pred_dfs = []
    
    for i, dset in enumerate(datasets):
        dset.load_or_generate_data()
        
        model = FeaturePredictor(dset, model_args=model_config, feature_model_args=fmodel_args)
        model.fit(dset, verbose=False)
        
        print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name, logs=dset.logs_args['which_logs'])
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [13]:
preds_df_3 = pred_results_df(3, make_params_usable(best_params_3))
preds_df_3.to_csv('logs_xgb_results_train_sets3_ALL.csv')

Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13034,)), ('top', (13034,)), ('base', (13034,)), ('logs', (13034, 11))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data


...

Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('logs', (10906, 11))]
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (3881,)), ('top', (3881,)), ('base', (3881,)), ('logs', (3881, 11))]
Training model for feature:  logs
predict_proba:  [[0.01891884 0.02778047 0.03864559 0.07464727 0.03545498 0.8045528 ]
 [0.01891884 0.02778047 0.03864559 0.07464727 0.03545498 0.8045528 ]
 [0.01920899 0.02820654 0.03923829 0.07579212 0.03637385 0.8011802 ]
 ...
 [0.0226778  0.65429443 0.03052688 0.07744154 0.02904843 0.18601096]
 [0.02332472 0.6328423  0.0313977  0.0796507  0.02987708 0.20290752]
 [0.02323416 0.63038516 0.03127579 0.08322408 0.02976108 0.2021197 ]]


In [14]:
preds_df_6 = pred_results_df(6, make_params_usable(best_params_6))
preds_df_6.to_csv('logs_xgb_results_train_sets6_ALL.csv')

Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13034,)), ('top', (13034,)), ('base', (13034,)), ('logs', (13034, 11))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data


...

Adding NaN log:  DTS
Feature shapes:  [('depth', (1981,)), ('top', (1981,)), ('base', (1981,)), ('logs', (1981, 11))]
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-20-6a  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), ('logs', (3542, 11))]
Training model for feature:  logs
predict_proba:  [[0.00304909 0.08638444 0.16377863 0.08901837 0.60956305 0.0482064 ]
 [0.00304033 0.08613627 0.16330814 0.08876265 0.6078119  0.05094066]
 [0.00304033 0.08613627 0.16330814 0.08876265 0.6078119  0.05094066]
 ...
 [0.00460944 0.31660894 0.19299147 0.20266075 0.24920578 0.03392363]
 [0.00459796 0.31582037 0.19251078 0.20215599 0.24858509 0.03632985]
 [0.00459195 0.31540763 0.1922592  0.20189181 0.24826023 0.03758918]]


In [15]:
preds_df_9 = pred_results_df(9, make_params_usable(best_params_9))
preds_df_9.to_csv('logs_xgb_results_train_sets9_ALL.csv')

Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9598,)), ('top', (9598,)), ('base', (9598,)), ('logs', (9598, 11))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('logs', (11210, 11))]
Loading Well:  204-19-7  from  /home/ross/Dropbox/core_data/facies/train_data


...

Adding NaN log:  DTS
Feature shapes:  [('depth', (9598,)), ('top', (9598,)), ('base', (9598,)), ('logs', (9598, 11))]
Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('logs', (3105, 11))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('logs', (11210, 11))]
Training model for feature:  logs
predict_proba:  [[0.05987606 0.0971621  0.12958457 0.21380678 0.22030494 0.2792656 ]
 [0.0602763  0.09781157 0.13045077 0.20855147 0.22177756 0.28113234]
 [0.06340107 0.09990217 0.13721342 0.1768707  0.23327465 0.28933802]
 ...
 [0.0576051  0.62631506 0.10505621 0.07183429 0.06963823 0.06955111]
 [0.0576051  0.62631506 0.10505621 0.07183429 0.06963823 0.06955111]
 [0.05755584 0.6257795  0.10496638 0.07262794 0.06957868 0.06949164]]


In [12]:
preds_df_3.shape, preds_df_6.shape, preds_df_9.shape

((152880, 25), (108025, 25), (71560, 25))

In [13]:
preds_df_3.head()

,base,confidence,depth,proba_0,proba_1,proba_2,proba_3,proba_4,regression,top,...,NEUT,PEF,RDEP,RSHAL,DTC,DTS,DTS1,DTS2,well,set_number
0,2208.085920,0.558097,2208.083152,0.162600,0.080687,0.558097,0.190883,0.007732,1.660552,2208.080550,...,-1.084784,-1.730394,1.346995,1.233757,1.417721,0.0,-0.148382,-0.483111,204-19-6,0
1,2208.091290,0.557830,2208.088522,0.162522,0.080648,0.557830,0.190792,0.008208,1.661818,2208.085920,...,-1.104229,-1.742244,1.349403,1.233453,1.421162,0.0,-0.136087,-0.474946,204-19-6,0
2,2208.096661,0.596532,2208.093892,0.157963,0.075003,0.596532,0.162869,0.007634,1.644650,2208.091290,...,-1.123674,-1.754094,1.351810,1.233149,1.424602,0.0,-0.123793,-0.466782,204-19-6,0
3,2208.102031,0.596227,2208.099262,0.158082,0.075059,0.596227,0.162992,0.007639,1.644456,2208.096661,...,-1.142953,-1.765778,1.354180,1.232846,1.428043,0.0,-0.111576,-0.458599,204-19-6,0
4,2208.107401,0.497442,2208.104632,0.193944,0.099274,0.497442,0.199968,0.009372,1.577861,2208.102031,...,-1.147980,-1.763182,1.353276,1.232685,1.431540,0.0,-0.106107,-0.448906,204-19-6,0


In [14]:
preds_df_3.columns

Index(['base', 'confidence', 'depth', 'proba_0', 'proba_1', 'proba_2',
       'proba_3', 'proba_4', 'regression', 'top', 'y_pred', 'y_true', 'GR',
       'SP', 'DENS', 'NEUT', 'PEF', 'RDEP', 'RSHAL', 'DTC', 'DTS', 'DTS1',
       'DTS2', 'well', 'set_number'],
      dtype='object')